In [3]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pymongo
from pprint import pprint

In [4]:
client = pymongo.MongoClient()
db = client.metrics

In [5]:
def group_by(df, bycols, agg_map):
    """

    @param df:      DataFrame
    @param bycols:  str or list
                        Column(s) to group by
    @param agg_map: dictionary or list of 2-tuples
                        Mapping from column to aggregate function e.g. [("city", "count"), ("salary", "mean"]
    @return:        DataFrame
                        Flattened dataframe, with multi-level index removed
    """
    grps = []
    if type(bycols) == str:
        bycols = [bycols]

    if type(agg_map) == dict:
        agg_map = agg_map.items()

    for k,v in agg_map:
        grp = df[bycols + [k]].groupby(bycols, ).agg(v)
        grp.reset_index(inplace=True)
        grp["%s(%s)" % (v,k)] = grp[k]
        del grp[k]
        grps.append(grp)

    m = grps[0]
    for grp in grps[1:]:
        m = pd.merge(m, grp, on=bycols, how="inner")
    return m

In [6]:
from bson.son import SON # needed to ensure dictionary is ordered (python default is not)
import hashlib

def hash_feats(fts):
    vals = fts.values
    joined = "|".join(map(lambda s: str(s),vals)).encode('utf-8') 
    return hashlib.sha224(joined).hexdigest()

def get_df_sorted_by_f1score(collection, params=None, filter_cols=True):
    if not params:
        params = []
    if type(params) == str:
        params = params.split(",")
    
    project = {
            "weighted_f1_score":"$WEIGHTED_MEAN_CONCEPT_CODES.f1_score",
            "micro_f1_score":  "$MICRO_F1.f1_score",
            "micro_recall":    "$MICRO_F1.recall",
            "micro_precision": "$MICRO_F1.precision",
    
    # PARAMETERS            
            "window_size":    "$parameters.window_size",
            "feats":          "$parameters.extractors",
            "count": {        "$size" : "$parameters.extractors" },
            "asof" :          "$asof",
            "_id":1
    }
    
    # No count for HMM
    if "_hmm" in collection.lower():
        del project["count"]
    
    for param in params:
        project[param] = "$parameters." + param

    feats_pipeline = [{
        "$project": project
    },
    {
        "$match":{
            "micro_f1_score": { "$exists" : True }        
        }
    },
    {
        "$sort":{
            "micro_f1_score": -1
        }
    },
    ]
    
    rows = [row for row in db[collection].aggregate(feats_pipeline)]
    df = pd.DataFrame(rows).sort_values("micro_f1_score", ascending=False)
    if params:
        df["hs_params"] = df[params].apply(hash_feats, axis=1)
        
    if filter_cols:
        cols = ["micro_f1_score", "micro_recall" ,"micro_precision" ] + params
        return df[cols]
    return df

In [16]:
from Metrics import rpf1a_from_tp_fp_tn_fn
from collections import defaultdict

def tally_counts(r, filter):
    tally = defaultdict(int)
    for k,v in r.items():
        if filter(k):
            for prop in "tp,tn,fp,fn".split(","):
                tally[prop] += v[prop]
    return tally

def get_causal_relation_metrics(collection, params, include_concept_codes=True):
    dicts = []
    for r in db[collection].find({}):
        d = {}
        cr_counts = tally_counts(r, lambda c: "->" in c)
        (rec, p, cr_f1, a) = rpf1a_from_tp_fp_tn_fn(cr_counts["tp"],cr_counts["fp"],cr_counts["tn"],cr_counts["fn"])
        d["cr_micro_f1"] = cr_f1
        d["cr_micro_rec"]  = rec
        d["cr_micro_prec"] = p
        if include_concept_codes:
            concept_counts = tally_counts(r, lambda c: c[0].isdigit())
            (rec, p, concept_f1, a) = rpf1a_from_tp_fp_tn_fn(concept_counts["tp"],concept_counts["fp"],concept_counts["tn"],concept_counts["fn"])
            d["concept_micro_f1"] = concept_f1
            d["concept_micro_rec"]  = rec
            d["concept_micro_prec"] = p
        parms = r["parameters"]
        for p in params:
            d[p] = parms[p]
        dicts.append(d)
    df = pd.DataFrame(dicts)
    fields = ("cr_micro_f1,cr_micro_rec,cr_micro_prec,concept_micro_f1,concept_micro_rec,concept_micro_prec," + ",".join(params)).split(",")
    if not include_concept_codes:
        fields = [f for f in fields if "concept" not in f]
    return df[fields].sort_values("cr_micro_f1", ascending=False)

In [8]:
def round_data(df, places=3):
    df_copy = df.copy()
    fmt_str = "{0:." + str(places) + "f}"
    cols = set([v for v in df_copy.columns.values if "micro_" in v])
    for c in cols:
        df_copy[c] = df[c].apply(lambda d: fmt_str.format(d))  
    return df_copy

# Binary Relevance - CR and Concept Codes

## Coral Bleaching
(no skin cancer results for this experiment)

In [12]:
params = "merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size"
df = get_causal_relation_metrics("CR_CB_TAGGING_TD_RNN", params.split(","))
df

,cr_micro_f1,cr_micro_rec,cr_micro_prec,concept_micro_f1,concept_micro_rec,concept_micro_prec,merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size
5,0.396690,0.626529,0.290224,0.583708,0.852237,0.443856,sum,2,True,True,256
4,0.305912,0.328199,0.286459,0.517585,0.661155,0.425244,sum,2,True,True,128
2,0.262015,0.730794,0.159622,0.595313,0.944704,0.434585,sum,1,True,True,256
1,0.214780,0.278353,0.174846,0.489831,0.699067,0.376994,sum,1,True,True,128
0,0.122902,0.118921,0.127158,0.517211,0.632040,0.437691,sum,1,True,True,64
3,0.000000,0.000000,0.000000,0.389768,0.399760,0.380264,sum,2,True,True,64


In [13]:
params = "merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size"
df = get_causal_relation_metrics("CR_CB_TAGGING_VD_RNN", params.split(","))
df.sort_values("cr_micro_f1", ascending=False)

,cr_micro_f1,cr_micro_rec,cr_micro_prec,concept_micro_f1,concept_micro_rec,concept_micro_prec,merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size
5,0.375512,0.594810,0.274359,0.574305,0.837298,0.437034,sum,2,True,True,256
4,0.282257,0.298736,0.267501,0.512915,0.650683,0.423292,sum,2,True,True,128
2,0.249717,0.697937,0.152062,0.595039,0.942434,0.434775,sum,1,True,True,256
1,0.213925,0.274451,0.175271,0.491845,0.702050,0.378513,sum,1,True,True,128
0,0.114974,0.112774,0.117260,0.519371,0.636893,0.438464,sum,1,True,True,64
3,0.000000,0.000000,0.000000,0.388121,0.397647,0.379040,sum,2,True,True,64


# Most Common Tag

## Coral Bleaching

In [28]:
params = "merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size"

### Training

In [35]:
df = get_df_sorted_by_f1score("CR_CB_TAGGING_TD_COMMON_TAG", params)
round_data(df,4)

,micro_f1_score,micro_recall,micro_precision,merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size
0,0.8453,0.7775,0.9259,sum,2,True,True,256
1,0.8014,0.7305,0.8875,sum,1,True,True,256
2,0.7773,0.7145,0.8521,sum,2,True,True,128
3,0.7339,0.6775,0.8005,sum,2,True,True,64
4,0.7219,0.6508,0.8103,sum,1,True,True,128
5,0.6906,0.6081,0.7990,sum,1,True,True,64
6,0.5394,0.4494,0.6744,sum,1,True,False,64


### Test

In [36]:
df = get_df_sorted_by_f1score("CR_CB_TAGGING_VD_COMMON_TAG", params)
round_data(df,4)

,micro_f1_score,micro_recall,micro_precision,merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size
0,0.6064,0.5342,0.7013,sum,2,True,True,256
1,0.5998,0.5325,0.6867,sum,2,True,True,128
2,0.5787,0.5126,0.6644,sum,1,True,True,256
3,0.5736,0.5187,0.6416,sum,2,True,True,64
4,0.5718,0.5029,0.6626,sum,1,True,True,128
5,0.5580,0.4783,0.6694,sum,1,True,True,64
6,0.4317,0.3485,0.5672,sum,1,True,False,64


## Skin Cancer

### Training

In [37]:
df = get_df_sorted_by_f1score("CR_SC_TAGGING_TD_COMMON_TAG", params)
round_data(df,4)

,micro_f1_score,micro_recall,micro_precision,merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size
0,0.8400,0.7805,0.9093,sum,2,True,True,128
1,0.8255,0.7685,0.8917,sum,2,True,True,256
2,0.8198,0.7676,0.8796,sum,1,True,True,256
3,0.7970,0.7428,0.8598,sum,2,True,True,64
4,0.7865,0.7229,0.8623,sum,1,True,True,64
5,0.7816,0.7127,0.8652,sum,1,True,True,128


### Test

In [38]:
df = get_df_sorted_by_f1score("CR_SC_TAGGING_VD_COMMON_TAG", params)
round_data(df,4)

,micro_f1_score,micro_recall,micro_precision,merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size
0,0.6959,0.6377,0.7657,sum,2,True,True,256
1,0.6908,0.6312,0.7627,sum,2,True,True,128
2,0.6784,0.6264,0.7398,sum,1,True,True,256
3,0.6750,0.6208,0.7396,sum,2,True,True,64
4,0.6705,0.6027,0.7555,sum,1,True,True,128
5,0.6663,0.6005,0.7482,sum,1,True,True,64
